In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## WQ_SAT
from wq_sat import config
from wq_sat.satellites import sentinel2
from wq_sat.utils import geo_utils

In [2]:
def get_bands(tiles, coordinates=None):
    
    tile_path = config.acolite_path(tiles[0])
    data_bands, coord = sentinel2.Rs_acolite(tile_path, coordinates)
    h, w = data_bands['B2'].shape

    ## Bands
    Rs492 = np.zeros((h, w, len(tiles)))
    Rs559 = np.zeros((h, w, len(tiles)))
    Rs665 = np.zeros((h, w, len(tiles)))
    Rs704 = np.zeros((h, w, len(tiles)))
    Rs842 = np.zeros((h, w, len(tiles)))
    chl = np.zeros((h, w, len(tiles)))
    
    for t, tile in enumerate(tiles):

        tile_path = config.acolite_path(tile)
        data_bands, coord = sentinel2.Rs_acolite(tile_path, coordinates)

        ## Bands
        Rs492[:,:,t] = data_bands['B2']
        Rs559[:,:,t] = data_bands['B3']
        Rs665[:,:,t] = data_bands['B4']
        Rs704[:,:,t] = data_bands['B5']
        Rs842[:,:,t] = data_bands['B8']
        chl[:,:,t] = data_bands['chl']

    ## Bands quality proxies
    Rs492_c = np.nanmax(Rs492, axis=2) # Rs492 -> B2
    Rs559_c = np.nanmax(Rs559, axis=2) # Rs559 -> B3
    Rs665_c = np.nanmax(Rs665, axis=2) # Rs665 -> B4
    Rs704_c = np.nanmax(Rs704, axis=2) # Rs704 -> B5
    Rs842_c = np.nanmax(Rs842, axis=2) # Rs842 -> B8
    chl_c = np.nanmax(chl, axis=2) # chl
    
    return Rs492_c, Rs559_c, Rs665_c, Rs704_c, Rs842_c, chl_c

In [11]:
region = 'Muro'
year = '2023'

path = os.path.join(config.data_path(), 'bathymetries', region, 'InSitu')
file = '{}_Insitu_bathymetry_EPSG32631_{}.tif'.format(region, year)

bathymetry, BBox, crs = geo_utils.load_geotiff(os.path.join(path, file))
mx = np.ma.masked_invalid(bathymetry)

# ROI
## transform coordinates
new_ul = geo_utils.transform_coordinates(BBox[0], inputEPSG=crs, outputEPSG=4326)
new_lr = geo_utils.transform_coordinates(BBox[1], inputEPSG=crs, outputEPSG=4326)
roi_coord = {'N': new_ul[0], 'W': new_ul[1], 'S': new_lr[0], 'E': new_lr[1]}

## tiles
tiles = config.get_tiles(region, year)

## Rs Bands
Rs492, Rs559, Rs665, Rs704, Rs842, chl = get_bands(tiles, roi_coord)
Rs492[mx.mask] = np.nan
Rs559[mx.mask] = np.nan
Rs665[mx.mask] = np.nan
Rs704[mx.mask] = np.nan
Rs842[mx.mask] = np.nan
chl[mx.mask] = np.nan

df_arr = np.vstack((bathymetry.flatten(),
                    Rs492.flatten(), 
                    Rs559.flatten(), 
                    Rs665.flatten(), 
                    Rs704.flatten(), 
                    Rs842.flatten(),
                    chl.flatten()))

df_arr = np.transpose(df_arr)
df = pd.DataFrame(df_arr, 
                  columns = ['Bathymetry', 'B2','B3','B4', 'B5', 'B8', 'chl'])

var = ['B2','B3','B4', 'B5', 'B8']
for v in var:
    print('Band: {} -> p1: {}'.format(v, np.nanpercentile(df[v], 1)))
    df[v][df[v] < np.nanpercentile(df[v], 1)] = np.nan
    
## Ratios transform
df['Rt23'] = np.log(3140*df['B2'])/np.log(3140*df['B3'])
df['Rt24'] = np.log(3140*df['B2'])/np.log(3140*df['B4'])
df['Rt28'] = np.log(3140*df['B2'])/np.log(3140*df['B8'])
df['Rt34'] = np.log(3140*df['B3'])/np.log(3140*df['B4'])
df['Rt38'] = np.log(3140*df['B3'])/np.log(3140*df['B8'])
df['Rt48'] = np.log(3140*df['B4'])/np.log(3140*df['B8'])

## Lyzenga transform
df['Lt2'] = np.log(df['B2'] - np.nanmin(df['B2']))
df['Lt3'] = np.log(df['B3'] - np.nanmin(df['B3']))
df['Lt4'] = np.log(df['B4'] - np.nanmin(df['B4']))
df['Lt8'] = np.log(df['B8'] - np.nanmin(df['B8']))

df = df.replace([-np.inf], np.nan)
df.loc[df.isnull().any(axis=1), :] = np.nan

output_path = '/home/wq_sat/notebooks/bathymetries/ML_SDB/data'
df.to_csv(os.path.join(output_path, '{}_{}.csv'.format(region, year)), index = None, header=True)

Loading /home/wqsat_data/Sentinel-2/ACOLITE/S2B_MSIL1C_20230325T102649_N0509_R108_T31TEE_20230325T122948
Selected pixel region: xmin=1000, ymin=9257, xmax=1109, ymax=9350:
Image size: width=110 x height=94
Loading /home/wqsat_data/Sentinel-2/ACOLITE/S2B_MSIL1C_20230325T102649_N0509_R108_T31TEE_20230325T122948
Selected pixel region: xmin=1000, ymin=9257, xmax=1109, ymax=9350:
Image size: width=110 x height=94
Loading /home/wqsat_data/Sentinel-2/ACOLITE/S2B_MSIL1C_20230417T103629_N0509_R008_T31TEE_20230417T123856
Selected pixel region: xmin=1000, ymin=9257, xmax=1109, ymax=9350:
Image size: width=110 x height=94
Loading /home/wqsat_data/Sentinel-2/ACOLITE/S2A_MSIL1C_20230320T102731_N0509_R108_T31TEE_20230320T141015
Selected pixel region: xmin=1000, ymin=9257, xmax=1109, ymax=9350:
Image size: width=110 x height=94
Loading /home/wqsat_data/Sentinel-2/ACOLITE/S2A_MSIL1C_20230323T103711_N0509_R008_T31TEE_20230323T173840
Selected pixel region: xmin=1000, ymin=9257, xmax=1109, ymax=9350:
Imag

/tmp/ipykernel_142864/330713726.py:29: RuntimeWarning: All-NaN slice encountered
  Rs492_c = np.nanmax(Rs492, axis=2) # Rs492 -> B2
/tmp/ipykernel_142864/330713726.py:30: RuntimeWarning: All-NaN slice encountered
  Rs559_c = np.nanmax(Rs559, axis=2) # Rs559 -> B3
/tmp/ipykernel_142864/330713726.py:31: RuntimeWarning: All-NaN slice encountered
  Rs665_c = np.nanmax(Rs665, axis=2) # Rs665 -> B4
/tmp/ipykernel_142864/330713726.py:32: RuntimeWarning: All-NaN slice encountered
  Rs704_c = np.nanmax(Rs704, axis=2) # Rs704 -> B5
/tmp/ipykernel_142864/330713726.py:33: RuntimeWarning: All-NaN slice encountered
  Rs842_c = np.nanmax(Rs842, axis=2) # Rs842 -> B8
/tmp/ipykernel_142864/330713726.py:34: RuntimeWarning: All-NaN slice encountered
  chl_c = np.nanmax(chl, axis=2) # chl
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/

## join data from the same regions

In [17]:
path = '/home/wq_sat/notebooks/bathymetries/ML_SDB/data'
region = 'SanBou'

df2018 = pd.read_csv(os.path.join(path, '{}_2017.csv'.format(region)))
df2018.loc[df2018.isnull().any(axis=1), :] = np.nan
df2018 = df2018.dropna()

df2019 = pd.read_csv(os.path.join(path, '{}_2021.csv'.format(region)))
df2019.loc[df2019.isnull().any(axis=1), :] = np.nan
df2019 = df2019.dropna()

df2020 = pd.read_csv(os.path.join(path, '{}_2022.csv'.format(region)))
df2020.loc[df2020.isnull().any(axis=1), :] = np.nan
df2020 = df2020.dropna()

df = pd.concat([df2018, df2019, df2020], axis = 0)

output_path = '/home/wq_sat/notebooks/bathymetries/ML_SDB/data'
df.to_csv(os.path.join(output_path, '{}.csv'.format(region)), index = None, header=True)

In [21]:
path = '/home/wq_sat/notebooks/bathymetries/ML_SDB/data'
region = 'Muro'

df2018 = pd.read_csv(os.path.join(path, '{}_2023.csv'.format(region)))
df2018.loc[df2018.isnull().any(axis=1), :] = np.nan
df2018 = df2018.dropna()

output_path = '/home/wq_sat/notebooks/bathymetries/ML_SDB/data'
df2018.to_csv(os.path.join(output_path, '{}.csv'.format(region)), index = None, header=True)

## join all data

In [31]:
path = '/home/wq_sat/notebooks/bathymetries/ML_SDB/data'

df_CalaMillor = pd.read_csv(os.path.join(path, 'CalaMillor.csv'))

df_SanBou = pd.read_csv(os.path.join(path, 'SanBou.csv'))

df_Muro = pd.read_csv(os.path.join(path, 'Muro.csv'))

df = pd.concat([df_CalaMillor, df_SanBou, df_Muro], axis = 0)

output_path = '/home/wq_sat/notebooks/bathymetries/ML_SDB/data'
df.to_csv(os.path.join(output_path, 'bathymetry_data.csv'), index = None, header=True)

## ML data analysis

In [32]:
df

,Bathymetry,B2,B3,B4,B5,B8,chl,Rt23,Rt24,Rt28,Rt34,Rt38,Rt48,Lt2,Lt3,Lt4,Lt8
0,-1.693478,0.036578,0.041910,0.016526,0.013325,0.004232,2.801855,0.972110,1.201177,1.833784,1.235639,1.886396,1.526656,-3.563276,-3.327024,-4.205634,-5.723626
1,-1.995529,0.035142,0.040581,0.019217,0.023148,0.007134,2.801855,0.970312,1.147215,1.512882,1.182316,1.559171,1.318743,-3.615255,-3.364747,-4.039726,-5.088084
2,-2.253562,0.034164,0.038859,0.031123,0.023148,0.018586,3.304615,0.973198,1.020346,1.149693,1.048447,1.181356,1.126767,-3.652253,-3.415842,-3.523085,-4.038568
3,-2.535824,0.029768,0.033131,0.019217,0.023148,0.007134,3.029637,0.976955,1.106733,1.459497,1.132840,1.493924,1.318743,-3.838041,-3.607486,-4.039726,-5.088084
4,-2.546011,0.028400,0.034890,0.031123,0.032117,0.016664,3.081493,0.956177,0.980016,1.134718,1.024931,1.186724,1.157857,-3.903656,-3.544633,-3.523085,-4.154093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7598,-3.933648,0.020926,0.022472,0.004597,0.003631,0.003515,2.789381,0.983246,1.567663,1.742864,1.594376,1.772562,1.111759,-4.335850,-4.129356,-5.698461,-5.910617
7599,-2.338984,0.038383,0.045474,0.011987,0.005119,0.001412,3.191955,0.965833,1.320785,3.217168,1.367508,3.330977,2.435801,-3.488441,-3.241771,-4.533727,-7.405996
7600,-2.705085,0.035469,0.042051,0.009465,0.004760,0.002276,3.210942,0.965143,1.389495,2.396329,1.439678,2.482875,1.724605,-3.588689,-3.333397,-4.801359,-6.521518
7601,-3.022751,0.031940,0.038001,0.008251,0.004760,0.002628,3.273729,0.963662,1.415908,2.183537,1.469299,2.265875,1.542147,-3.725347,-3.453912,-4.961234,-6.307234


In [57]:
df_bands = df[['Bathymetry', 'B2', 'B3', 'B4', 'B5', 'B8', 'chl']]

# Correlations Matrix
corr = df_bands[df_bands['Bathymetry'] <= -5].corr()

# Fill diagonal and upper half with NaNs
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(color='#f1f1f1')  # Color NaNs grey
 .format(precision=2))

,Bathymetry,B2,B3,B4,B5,B8,chl
Bathymetry,nan,nan,nan,nan,nan,nan,nan
B2,0.56,nan,nan,nan,nan,nan,nan
B3,0.68,0.95,nan,nan,nan,nan,nan
B4,0.31,0.43,0.55,nan,nan,nan,nan
B5,0.33,0.36,0.48,0.85,nan,nan,nan
B8,0.17,0.31,0.35,0.74,0.67,nan,nan
chl,0.71,0.54,0.71,0.54,0.53,0.20,nan


In [58]:
df_ratios = df[['Bathymetry', 'Rt23', 'Rt24', 'Rt28', 'Rt34', 'Rt38', 'Rt48']]

# Correlations Matrix
corr = df_ratios[df_ratios['Bathymetry'] <= -5].corr()

# Fill diagonal and upper half with NaNs
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(color='#f1f1f1')  # Color NaNs grey
 .format(precision=2))

,Bathymetry,Rt23,Rt24,Rt28,Rt34,Rt38,Rt48
Bathymetry,nan,nan,nan,nan,nan,nan,nan
Rt23,-0.79,nan,nan,nan,nan,nan,nan
Rt24,-0.21,0.45,nan,nan,nan,nan,nan
Rt28,-0.16,0.22,0.50,nan,nan,nan,nan
Rt34,-0.01,0.21,0.97,0.48,nan,nan,nan
Rt38,-0.03,0.07,0.45,0.99,0.48,nan,nan
Rt48,-0.03,-0.08,-0.09,0.78,-0.08,0.79,nan


In [59]:
df_lyzenga = df[['Bathymetry', 'Lt2', 'Lt3', 'Lt4', 'Lt8']]

# Correlations Matrix
corr = df_lyzenga[df_lyzenga['Bathymetry'] <= -5].corr()

# Fill diagonal and upper half with NaNs
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(color='#f1f1f1')  # Color NaNs grey
 .format(precision=2))

,Bathymetry,Lt2,Lt3,Lt4,Lt8
Bathymetry,nan,nan,nan,nan,nan
Lt2,0.49,nan,nan,nan,nan
Lt3,0.65,0.89,nan,nan,nan
Lt4,0.30,0.51,0.56,nan,nan
Lt8,0.14,0.30,0.29,0.39,nan
